In [93]:
import pandas as pd
from sqlalchemy import create_engine
import pymysql

### Store World Cup Matches CSV into DataFrame

In [119]:
csv_file = "../Resources/WorldCupMatches.csv"
WorldCupMatches_df = pd.read_csv(csv_file)
WorldCupMatches_df.tail()

,Year,Datetime,Stage,Stadium,City,Home Team Name,Home Team Goals,Away Team Goals,Away Team Name,Win conditions,Attendance,Half-time Home Goals,Half-time Away Goals,Referee,Assistant 1,Assistant 2,RoundID,MatchID,Home Team Initials,Away Team Initials
4567,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4568,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4569,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4570,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4571,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Create new data with select columns

In [95]:
#new_customer_data_df = customer_data_df[['id', 'first_name', 'last_name']].copy()
#new_customer_data_df.head()

### Store World Cup Players CSV into a DataFrame

In [120]:
csv_file = "../Resources/WorldCupPlayers.csv"
customer_data_df = pd.read_csv(csv_file)
customer_data_df.tail()

,RoundID,MatchID,Team Initials,Coach Name,Line-up,Shirt Number,Player Name,Position,Event
37779,255959,300186501,ARG,SABELLA Alejandro (ARG),N,19,ALVAREZ,NaN,NaN
37780,255959,300186501,GER,LOEW Joachim (GER),N,6,KHEDIRA,NaN,NaN
37781,255959,300186501,ARG,SABELLA Alejandro (ARG),N,20,AGUERO,NaN,IH46' Y65'
37782,255959,300186501,GER,LOEW Joachim (GER),N,21,MUSTAFI,NaN,NaN
37783,255959,300186501,ARG,SABELLA Alejandro (ARG),N,23,BASANTA,NaN,NaN


### Clean DataFrame

In [97]:
# new_customer_location_df = customer_location_df[["id", "address", "us_state"]].copy()
# new_customer_location_df.head()

In [121]:
csv_file = "../Resources/WorldCups.csv"
customer_data_df = pd.read_csv(csv_file)
customer_data_df.tail()

,Year,Country,Winner,Runners-Up,Third,Fourth,GoalsScored,QualifiedTeams,MatchesPlayed,Attendance
15,1998,France,France,Brazil,Croatia,Netherlands,171,32,64,2.785.100
16,2002,Korea/Japan,Brazil,Germany,Turkey,Korea Republic,161,32,64,2.705.197
17,2006,Germany,Italy,France,Germany,Portugal,147,32,64,3.359.439
18,2010,South Africa,Spain,Netherlands,Germany,Uruguay,145,32,64,3.178.856
19,2014,Brazil,Germany,Argentina,Netherlands,Brazil,171,32,64,3.386.810


In [ ]:
# new_customer_location_df = customer_location_df[["id", "address", "us_state"]].copy()
# new_customer_location_df.head()

In [122]:
csv_file = "../Resources/GLOBSES.csv"
customer_data_df = pd.read_csv(csv_file, encoding="ISO-8859-1") ## Had to rename from GLOB.SES and to
## and we had to determine the encoding character set
customer_data_df.tail()

,unid,wbid,country,year,SES,gdppc,yrseduc,popshare
2081,716,ZWE,Zimbabwe,1940,52.746567,813.0000,NaN,0.001113
2082,716,ZWE,Zimbabwe,2010,27.091389,1388.9730,NaN,0.002074
2083,716,ZWE,Zimbabwe,1990,59.110970,2526.0723,NaN,0.002039
2084,716,ZWE,Zimbabwe,1900,54.722645,601.0000,NaN,0.000996
2085,716,ZWE,Zimbabwe,1960,53.811077,938.0000,NaN,0.001268


In [ ]:
# new_customer_location_df = customer_location_df[["id", "address", "us_state"]].copy()
# new_customer_location_df.head()

### Connect to local database

In [98]:
## A password is required, a database is not required
conn = pymysql.connect(host='localhost',
                             user='root',
                             password='',
                             db='')

In [99]:
cursor = conn.cursor()

### Create Database

In [100]:
#cursor.execute('drop database etl_project')
cursor.execute('CREATE DATABASE etl_project')

ProgrammingError: (1007, "Can't create database 'etl_project'; database exists")

In [102]:
cursor.execute('USE etl_project')

0

In [89]:
# Create a table with an auto increment row as the primary key
cursor.execute('CREATE TABLE WorldCupMatches (matches INT NOT NULL AUTO_INCREMENT, PRIMARY KEY (matches))')
cursor.execute('CREATE TABLE WorldCupPlayers (players INT NOT NULL AUTO_INCREMENT, PRIMARY KEY (players))')
cursor.execute('CREATE TABLE WorldCups (world_cups INT NOT NULL AUTO_INCREMENT, PRIMARY KEY (world_cups))')

InternalError: (1050, "Table 'worldcupmatches' already exists")

In [101]:
cursor.execute("SHOW TABLES")

InternalError: (1046, 'No database selected')

In [103]:
result = cursor.fetchall()
for row in result:
    print(row)

In [104]:
conn.close()

In [105]:
## This is where I stopped.

### Use pandas to load csv converted DataFrame into database

In [60]:
new_customer_data_df.to_sql(name='WorldCupMatches_df', con=conn, if_exists='append', index=False)

DatabaseError: Execution failed on sql: SELECT name FROM sqlite_master WHERE type='table' AND name=?;
not all arguments converted during string formatting
unable to rollback

### Use pandas to load json converted DataFrame into database

In [23]:
new_customer_location_df.to_sql(name='customer_location', con=engine, if_exists='append', index=False)

### Confirm data has been added by querying the customer_name table
* NOTE: can also check using pgAdmin

In [24]:
pd.read_sql_query('select * from customer_name', con=engine).head()

,id,first_name,last_name
0,1,Benetta,Cancott
1,2,Lilyan,Cherry
2,3,Ezekiel,Benasik
3,4,Kennedy,Atlay
4,5,Sanford,Salmen


### Confirm data has been added by querying the customer_location table

In [27]:
pd.read_sql_query('select * from customer_location a join customer_name b on a.id = b.id', con=engine).head()

,id,address,us_state,id,first_name,last_name
0,1,043 Mockingbird Place,Indiana,1,Benetta,Cancott
1,1,043 Mockingbird Place,Indiana,1,Benetta,Cancott
2,2,4 Prentice Point,Indiana,2,Lilyan,Cherry
3,2,4 Prentice Point,Indiana,2,Lilyan,Cherry
4,3,46 Derek Junction,Texas,3,Ezekiel,Benasik


In [ ]:
pd.rea